This exercise will require you to pull some data from the Quandl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [15]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'MY_API_KEY_IS_SECRET'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [16]:
# First, import the relevant modules
import requests
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the "V3" quandl api at `https://www.quandl.com/api/v3/`

In [138]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json"
params = dict(limit=1, api_key=API_KEY)
res = requests.get(url, params=params)
print(res)

<Response [200]>


In [139]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json = res.json()
#print(json)

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

#### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [140]:
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json" # FSE - database name, AFX_X - dataset name
params = dict(start_date='2017-01-01', end_date='2017-12-31', api_key=API_KEY)
res = requests.get(url, params=params)
#print(res)
json = res.json()
#print(json)

#### 2. Convert the returned JSON object into a Python dictionary.

In [141]:
pydict = dict(json)

#### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [142]:
# See what columns are there
pydict['dataset']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [143]:
data = pydict['dataset']['data'] # data for the year of 2017

In [144]:
OpenList = [] # list of opening prices
for date_prices in data: # iterating over dictionary
    OpenList.append(date_prices[1]) # keep adding opening price values at different dates
#print(OpenList) # some values are None
OpenList = list( filter(None.__ne__, OpenList) ) # filter out None values and convert back to list
print("Highest opening price = $" + str(max(OpenList)))
print("Lowest opening price = $" + str(min(OpenList)))

Highest opening price = $53.11
Lowest opening price = $34.0


#### 4. What was the largest change in any one day (based on High and Low price)?

In [145]:
AmplitudeList = [] # list of high minus low prices
for date_prices in data: # iterating over dictionary
    AmplitudeList.append(date_prices[2] - date_prices[3]) # keep adding price change values at different dates
#print(AmplitudeList) # all values are OK
print("Largest change = $%.2f" % max(AmplitudeList)) # formatted output to suppress rounding error

Largest change = $2.81


#### 5. What was the largest change between any two days (based on Closing Price)?

In [146]:
CloseList = [] # list of closing prices
for date_prices in data: # iterating over dictionary
    CloseList.append(date_prices[4]) # keep adding closing price values at different dates
#print(CloseList) # all values are OK
ChangeList = [] # list of changes in closing price
for i in range(1,len(CloseList)):
    ChangeList.append( CloseList[i] - CloseList[i-1] ) # keep adding closing price change values between two consecutive dates
print("Largest change in closing price = $%g" % max(ChangeList)) # suppress rounding error using %g format

Largest change in closing price = $2.56


#### 6. What was the average daily trading volume during this year?

In [147]:
TradevolumeList = [int(tv[6]) for tv in data] # extract daily trading volumes using list comprehension
print("Average daily trading volume during 2017 = $%.2f" % (sum(TradevolumeList)/len(TradevolumeList)))

Average daily trading volume during 2017 = $89124.34


#### 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [148]:
def median_list(lst):
    '''Median of a list.'''
    l = sorted(lst) # sort the list (order does not matter)
    n = len(l) # no. of elements in the list
    if n % 2 == 1: # if odd no. of elements
        return l[n//2]
    else: # if even no. of elements
        return 0.5*(l[n//2-1] + l[n//2]) # arithmetic average of two central elements

In [149]:
print("Median daily trading volume during 2017 = $%.2f" % median_list(TradevolumeList))

Median daily trading volume during 2017 = $76286.00
